In [16]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [17]:
train_set= ('./data_pre/mel_spectrogram_imgs/train')
test_set=('./data_pre/mel_spectrogram_imgs/test')
val_set= (r'./data_pre/mel_spectrogram_imgs/val')

In [18]:
train_datagen = image.ImageDataGenerator(rescale= 1./255)
val_datagen= image.ImageDataGenerator(rescale= 1./255)
test_datagen= image.ImageDataGenerator(rescale= 1./255)


In [19]:

train_generator = train_datagen.flow_from_directory(train_set,batch_size =128 ,class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory(test_set,shuffle=True,batch_size =128 ,class_mode = 'categorical')
validation_generator = test_datagen.flow_from_directory(val_set,shuffle=True,batch_size =128 ,class_mode = 'categorical')

Found 699 images belonging to 10 classes.
Found 151 images belonging to 10 classes.
Found 149 images belonging to 10 classes.


In [20]:
x_train, y_train = next(train_generator)
x_val,y_val= next(validation_generator)
x_test, y_test = next(test_generator)

In [21]:
import optuna
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import swish, relu,selu,mish


def objective(trial):
    # Define the search space for hyperparameters
    num_layers = trial.suggest_int('num_layers', 0, 3)
    layer_size = trial.suggest_int('layer_size', 10, 500)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    use_batch_norm = trial.suggest_categorical('use_batch_norm', [True, False])
    activation = trial.suggest_categorical('activation', ['relu','swish','selu','mish'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)

    # Create the MLP model with hyperparameters
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the layers in the base model
    for layer in base_model.layers:
        layer.trainable = False

# Create a new model
    model = Sequential()

# Add the VGG16 base model to the new model
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(layer_size, input_shape=(x_train.shape[1],)))
    model.add(Activation(activation))
    model.add(Dropout(dropout_rate))
    
    for _ in range(num_layers):
        model.add(Dense(layer_size))
        if use_batch_norm:
            model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(Dropout(dropout_rate))
    
    model.add(Dense(10, activation='softmax'))

    # Compile the model with hyperparameters
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model with hyperparameters
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min', min_delta=0.001, restore_best_weights=True)
    pruning_callback = optuna.integration.TFKerasPruningCallback(trial, 'val_loss')
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, callbacks=[early_stopping, pruning_callback], verbose=1)
    
    # Return the validation accuracy as the objective value
    return history.history['val_accuracy'][-1]

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and objective value
best_params = study.best_params
best_value = study.best_value
print('Best Hyperparameters:', best_params)
print('Best Objective Value:', best_value)


[I 2024-04-19 20:26:54,076] A new study created in memory with name: no-name-eb9a6ee4-f8ac-4c8c-8c4a-1736928fc08f
/tmp/ipykernel_119327/1239521713.py:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
/tmp/ipykernel_119327/1239521713.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


Epoch 1/50
4/4 [==============================] - 2s 346ms/step - loss: 3.5734 - accuracy: 0.0469 - val_loss: 3.1283 - val_accuracy: 0.0859
Epoch 2/50
4/4 [==============================] - 1s 274ms/step - loss: 3.0070 - accuracy: 0.1562 - val_loss: 2.7903 - val_accuracy: 0.0859
Epoch 3/50
4/4 [==============================] - 1s 269ms/step - loss: 2.4486 - accuracy: 0.1875 - val_loss: 2.2148 - val_accuracy: 0.2109
Epoch 4/50
4/4 [==============================] - 1s 260ms/step - loss: 2.2750 - accuracy: 0.2109 - val_loss: 2.3994 - val_accuracy: 0.1641
Epoch 5/50
4/4 [==============================] - 1s 274ms/step - loss: 2.1571 - accuracy: 0.2344 - val_loss: 2.1800 - val_accuracy: 0.2109
Epoch 6/50
4/4 [==============================] - 1s 260ms/step - loss: 2.0724 - accuracy: 0.3125 - val_loss: 2.2311 - val_accuracy: 0.1953
Epoch 7/50
4/4 [==============================] - 1s 270ms/step - loss: 1.8206 - accuracy: 0.3672 - val_loss: 2.1232 - val_accuracy: 0.3281
Epoch 8/50
4/4 [====

[I 2024-04-19 20:27:41,132] Trial 0 finished with value: 0.421875 and parameters: {'num_layers': 0, 'layer_size': 491, 'dropout_rate': 0.468023770666822, 'use_batch_norm': True, 'activation': 'swish', 'learning_rate': 0.00018607437154878221}. Best is trial 0 with value: 0.421875.


Epoch 1/50
4/4 [==============================] - 3s 330ms/step - loss: 2.7366 - accuracy: 0.0625 - val_loss: 2.3124 - val_accuracy: 0.0859
Epoch 2/50
4/4 [==============================] - 1s 257ms/step - loss: 2.5775 - accuracy: 0.1172 - val_loss: 2.3141 - val_accuracy: 0.0859
Epoch 3/50
4/4 [==============================] - 1s 257ms/step - loss: 2.6383 - accuracy: 0.0469 - val_loss: 2.3161 - val_accuracy: 0.0859
Epoch 4/50
4/4 [==============================] - 1s 257ms/step - loss: 2.6673 - accuracy: 0.0938 - val_loss: 2.3166 - val_accuracy: 0.0859
Epoch 5/50
4/4 [==============================] - 1s 256ms/step - loss: 2.6284 - accuracy: 0.0938 - val_loss: 2.3161 - val_accuracy: 0.0859
Epoch 6/50
4/4 [==============================] - 1s 257ms/step - loss: 2.5843 - accuracy: 0.1016 - val_loss: 2.3162 - val_accuracy: 0.0859
Epoch 7/50
4/4 [==============================] - 1s 258ms/step - loss: 2.5154 - accuracy: 0.1094 - val_loss: 2.3184 - val_accuracy: 0.0859
Epoch 8/50
4/4 [====

[I 2024-04-19 20:27:52,850] Trial 1 finished with value: 0.0859375 and parameters: {'num_layers': 3, 'layer_size': 18, 'dropout_rate': 0.2724910391934923, 'use_batch_norm': True, 'activation': 'relu', 'learning_rate': 3.41487294462982e-05}. Best is trial 0 with value: 0.421875.


...

Epoch 1/50
4/4 [==============================] - 2s 331ms/step - loss: 2.9503 - accuracy: 0.0938 - val_loss: 3.2328 - val_accuracy: 0.1094
Epoch 2/50
4/4 [==============================] - 1s 266ms/step - loss: 2.8322 - accuracy: 0.1484 - val_loss: 3.0130 - val_accuracy: 0.1094
Epoch 3/50
4/4 [==============================] - 1s 267ms/step - loss: 2.8338 - accuracy: 0.0781 - val_loss: 2.7185 - val_accuracy: 0.1094
Epoch 4/50
4/4 [==============================] - 1s 259ms/step - loss: 2.8491 - accuracy: 0.1172 - val_loss: 2.7360 - val_accuracy: 0.1562
Epoch 5/50
4/4 [==============================] - 1s 259ms/step - loss: 2.7346 - accuracy: 0.1406 - val_loss: 2.7538 - val_accuracy: 0.0859
Epoch 6/50
4/4 [==============================] - 1s 258ms/step - loss: 2.6923 - accuracy: 0.0547 - val_loss: 2.8041 - val_accuracy: 0.0859
Epoch 7/50
4/4 [==============================] - 1s 258ms/step - loss: 2.7382 - accuracy: 0.1172 - val_loss: 2.7483 - val_accuracy: 0.0859
Epoch 8/50
4/4 [====

[I 2024-04-19 20:29:45,528] Trial 9 finished with value: 0.0859375 and parameters: {'num_layers': 2, 'layer_size': 74, 'dropout_rate': 0.37166373933773855, 'use_batch_norm': True, 'activation': 'relu', 'learning_rate': 0.0017629486375693364}. Best is trial 0 with value: 0.421875.


Epoch 1/50


2024-04-19 20:29:56.120407: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 512.00MiB (rounded to 536870912)requested by op sequential_13/vgg16/block1_conv2/Relu
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-04-19 20:29:56.120622: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-04-19 20:29:56.120635: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 397, Chunks in use: 397. 99.2KiB allocated for chunks. 99.2KiB in use in bin. 13.0KiB client-requested in use in bin.
2024-04-19 20:29:56.120641: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 63, Chunks in use: 63. 35.5KiB allocated for chunks. 35.5KiB in use in bin. 27.9KiB client-requested in 

ResourceExhaustedError: Graph execution error:

Detected at node sequential_13/vgg16/block1_conv2/Relu defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/alinnavas/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/alinnavas/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/alinnavas/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/home/alinnavas/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/alinnavas/.local/lib/python3.10/site-packag